In [1]:
import gc
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from sklearn.cluster import AgglomerativeClustering


# Extracting the dataset

In [2]:
df = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1])
df

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00      NaN   
...           ...                  ...                  ...      ...   
155316          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155317          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155318          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155319          0  2017-03-30 15:20:35  2017-02-17 00:00:00      NaN   
155320          0  2017-03-26 16:22:18  2017-03-26 00:00:00      NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
2                4      1                                            <p></p>   
3                4      1                                            <p></p>   
5                4      1                                            <p></p>   
10               4      6                                                NaN   
20               2      4          <p> "spiritual songs" from Nicky Cook</p>   
...            ...    ...                                                ...   
155316           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155317           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155318           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155319           0  22940  <p>A live performance at Monty Hall on Feb 17,...   
155320           1  22906                                                NaN   

                               \
         listens     producer   
track_id                        
2           6073          NaN   
3           6073          NaN   
5           6073          NaN   
10         47632          NaN   
20          2710          NaN   
...          ...          ...   
155316      1506  Monty Hall    
155317      1506  Monty Hall    
155318      1506  Monty Hall    
155319      1506  Monty Hall    
155320      7481          NaN   

                                                             ...       track  \
                                                       tags  ... information   
track_id                                                     ...               
2                                                        []  ...         NaN   
3                                                        []  ...         NaN   
5                                                        []  ...         NaN   
10                                                       []  ...         NaN   
20                                                       []  ...         NaN   
...                                                     ...  ...         ...   
155316                                                   []  ...         NaN   
155317                                                   []  ...         NaN   
155318                                                   []  ...         NaN   
155319                                                   []  ...         NaN   
155320    ['ballad', 'epic', 'rockabilly', 'curse', 'hex...  ...         NaN   

                                 \
         interest language_code   
track_id                          
2            4656            en   
3            1470            en   
5            1933            en   
10          54881            en   
20            978            en   
...           ... 

In [3]:
#df.columns

In [4]:
small_data = df[df[('set','subset')] == 'small'].index.to_list()

In [5]:
data = pd.read_csv('data/features.csv', header=[0,1,2], index_col=0)
data.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
20         -0.289294 -0.816410  0.043851 -0.804761  ...  0.095003  0.022492   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
20          0.021355  16.669037  0.469727  0.047225  0.040039  0.000977   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
20          3.189831  0.030993  

[5 rows x 518 columns]

In [6]:
small_dataset = data[data.index.isin(small_data)]
small_dataset

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
140           0.533579 -0.623885 -1.086205 -1.081079 -0.765151 -0.072282   
141           0.172898 -0.284804 -1.169662 -1.062855 -0.706868 -0.708281   
...                ...       ...       ...       ...       ...       ...   
154308       -0.677709 -0.830608 -0.686122  0.185158  2.854806 -1.131797   
154309       -0.585059 -0.635075 -0.194742 -0.434809 -1.357310 -0.257745   
154413       -0.214509 -1.130469  0.718534 -0.368448 -0.147830 -0.099409   
154414       -0.487371 -0.923754 -0.283099 -0.435221 -1.137329 -0.798039   
155066        0.044216 -0.300441 -0.217022 -0.356106 -1.085789 -1.185135   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
140        -0.882913 -0.582376 -0.884749 -0.645214  ...  0.157683  0.028070   
141        -0.204884  0.023624 -0.642770 -0.786291  ...  0.145994  0.024342   
...              ...       ...       ...       ...  ...       ...       ...   
154308     -1.161222 -1.110237 -0.818111 -1.388484  ...  0.117287  0.021906   
154309     -0.639082 -0.572640 -1.225242  5.548715  ...  0.180397  0.029303   
154413     -1.325709 -0.105248 -1.363881  1.229534  ...  0.156587  0.036926   
154414     -0.258168  1.004049 -0.499121  0.746973  ...  0.181294  0.026723   
155066     -0.655948 -1.517006 -0.490595 -1.267883  ...  0.189816  0.029439   

feature                      zcr                                          \
statistics              kurtosis       max      mean    median       min   
number            06          01        01        01        01        01   
track_id                                                                   
2           0.012111    5.758890  0.459473  0.085629  0.071289  0.000000   
5           0.014759    6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921   21.434212  0.452148  0.077515  0.071777  0.000000   
140         0.025946   11.052547  0.379395  0.052379  0.036621  0.001953   
141         0.032111   32.994659  0.415527  0.040267  0.034668  0.002930   
...              ...         ...       ...       ...       ...       ...   
154308      0.021999   53.807804  0.769043  0.042997  0.022949  0.001953   
154309      0.023771   12.597441  0.768555  0.082979  0.037109  0.008789   
154413      0.038113   25.368595  0.323242  0.024532  0.018066  0.000977   
154414      0.030980   21.276468  0.511230  0.046116  0.033691  0.003418   
155066      0.025806  116.044044  0.554199  0.016058  0.009766  0.000000   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
5           2.193303  0.044861  
10          3.542325  0.040800  
140         3.143968  0.057712  
141         4.204097  0.028665  
...              ...       ...  
154308      6.928385  0.084298  
154309      3.441251  0.129480  
154413      3.736646  0.023821  
154414      3.997052  0.045733  
155066      9.688635  0.030787  

[8000 rows x 518 columns]

In [7]:
# clean memory
del data
del df
del small_data
gc.collect()

42

# 1.1 - Hierarchical Clustering

In [115]:
clustering_stats = pd.DataFrame(columns=['num_clusters', 'radius', 'diameter', 'density_r2',
                                'density_d2', 'avg_radius', 'avg_diameter', 'avg_density_r2', 'avg_density_d2'])

for i in range(8, 17):
    # Hierarchical clustering of the dataset for k = 8 to k = 16
    clustering = AgglomerativeClustering(n_clusters=i).fit_predict(small_dataset)
    clustered = small_dataset.copy()
    clustered['cluster'] = clustering
    
    # Calculating the centroids assuming euclidean distance was used
    centroids = clustered.groupby("cluster").mean().values
    
    radius = {}
    density_r2 = {}
    density_d2 = {}
    diameter = {}
    
    # Calculate the radius, diameter and density of each cluster 
    # and add it to a dictionary
    for x in range(0, len(centroids)):
        points = clustered[clustered[('cluster', '', '')] == x].values.tolist()
        # remove the cluster number from the list of points
        points = [x.__delitem__(-1) for x in points]
        calc = max([np.linalg.norm(centroids[x], point) for point in points])
        radius[x] = calc
        diameter[x] = radius[x] * 2
        density_r2[x] = len(points) / (radius[x] ** 2)
        density_d2[x] = len(points) / (diameter[x] ** 2)
    
    # Calculate avg metrics
    avg_radius = np.mean(list(radius.values()))
    avg_diameter = np.mean(list(diameter.values()))
    avg_density_r2 = np.mean(list(density_r2.values()))
    avg_density_d2 = np.mean(list(density_d2.values()))

    # Add the metrics of each number of clusters to a dataframe to be easier to compare them
    df_row = {'num_clusters': i, 'radius': radius, 'diameter': diameter, 'density_r2': density_r2,
            'density_d2': density_d2, 'avg_radius': avg_radius, 'avg_diameter': avg_diameter, 
            'avg_density_r2': avg_density_r2, 'avg_density_d2': avg_density_d2}
    df_row = pd.DataFrame(df_row)
    
    clustering_stats = pd.concat([clustering_stats, df_row], ignore_index=True)
    
    

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.

In [116]:
clustering_stats

,num_clusters,radius,diameter,density_r2,density_d2,avg_radius,avg_diameter,avg_density_r2,avg_density_d2
0,8,12682.664760,25365.329521,0.000011,2.828723e-06,12360.532264,24721.064527,0.000007,1.703661e-06
1,8,11961.375611,23922.751222,0.000015,3.713100e-06,12360.532264,24721.064527,0.000007,1.703661e-06
2,8,8159.365630,16318.731260,0.000006,1.396916e-06,12360.532264,24721.064527,0.000007,1.703661e-06
3,8,13676.988903,27353.977805,0.000005,1.129317e-06,12360.532264,24721.064527,0.000007,1.703661e-06
4,8,15095.211535,30190.423070,0.000001,3.554730e-07,12360.532264,24721.064527,0.000007,1.703661e-06
...,...,...,...,...,...,...,...,...,...
103,16,6069.934768,12139.869536,0.000003,6.785345e-07,12427.285131,24854.570263,0.000003,8.603169e-07
104,16,13392.593766,26785.187532,0.000001,3.164000e-07,12427.285131,24854.570263,0.000003,8.603169e-07
105,16,14864.427189,29728.854379,0.000001,3.088917e-07,12427.285131,24854.570263,0.000003,8.603169e-07
106,16,12776.436885,25552.873770,0.000004,1.029176e-06,12427.285131,24854.570263,0.000003,8.603169e-07


In [129]:
clustering_stats.loc[clustering_stats.num_clusters == 8]

,num_clusters,radius,diameter,density_r2,density_d2,avg_radius,avg_diameter,avg_density_r2,avg_density_d2
0,8,12682.664760,25365.329521,0.000011,2.828723e-06,12360.532264,24721.064527,0.000007,0.000002
1,8,11961.375611,23922.751222,0.000015,3.713100e-06,12360.532264,24721.064527,0.000007,0.000002
2,8,8159.365630,16318.731260,0.000006,1.396916e-06,12360.532264,24721.064527,0.000007,0.000002
3,8,13676.988903,27353.977805,0.000005,1.129317e-06,12360.532264,24721.064527,0.000007,0.000002
4,8,15095.211535,30190.423070,0.000001,3.554730e-07,12360.532264,24721.064527,0.000007,0.000002
5,8,12304.453667,24608.907335,0.000011,2.660179e-06,12360.532264,24721.064527,0.000007,0.000002
6,8,14492.807949,28985.615899,0.000002,5.522719e-07,12360.532264,24721.064527,0.000007,0.000002
7,8,10511.390054,21022.780107,0.000004,9.933087e-07,12360.532264,24721.064527,0.000007,0.000002


# 1.2 BRF Implementation

In [ ]:
sc = SparkContext(appName="Assignment2_E1")
